# Census data case study

Import libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Load data set

In [4]:
census_train = pd.read_excel('Copy of censusTrain.xlsx', index_col = 0)
census_test = pd.read_excel('Copy of censustest.xlsx', index_col = 0)
test = census_test

In [5]:
# train and test data set info, notice missing values, 6 numerical columns, 9 catigorical columns 
census_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32561 entries, 1 to 32561
Data columns (total 15 columns):
age               32561 non-null int64
work_class        30725 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        30718 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    31978 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 4.0+ MB


In [6]:
census_train['income'].value_counts()

<=50K    24720
>50K      7841
Name: income, dtype: int64

In [7]:
census_train.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [8]:
# test data set has features only with missing values
census_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16281 entries, 32562 to 48842
Data columns (total 14 columns):
age               16281 non-null int64
work_class        15318 non-null object
fnlwgt            16281 non-null int64
education         16281 non-null object
education_num     16281 non-null int64
marital_status    16281 non-null object
occupation        15315 non-null object
relationship      16281 non-null object
race              16281 non-null object
sex               16281 non-null object
capital_gain      16281 non-null int64
capital_loss      16281 non-null int64
hours_per_week    16281 non-null int64
native_country    16007 non-null object
dtypes: int64(6), object(8)
memory usage: 1.9+ MB


In [9]:
# Label the target column with 0 and 1
census_train['income'] = census_train['income'].apply(lambda x: 1 if x=='>50K' else 0)

In [10]:
census_train.head()

,age,work_class,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
id,,,,,,,,,,,,,,,
1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


Deal with missing data:

In [11]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self
    
    def transform(self, X, y=None):
        return X.fillna(self.fill)
    

census_train = DataFrameImputer().fit_transform(census_train)
census_test = DataFrameImputer().fit_transform(census_test)

In [12]:
census_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32561 entries, 1 to 32561
Data columns (total 15 columns):
age               32561 non-null int64
work_class        32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income            32561 non-null int64
dtypes: int64(7), object(8)
memory usage: 4.0+ MB


In [13]:
census_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16281 entries, 32562 to 48842
Data columns (total 14 columns):
age               16281 non-null int64
work_class        16281 non-null object
fnlwgt            16281 non-null int64
education         16281 non-null object
education_num     16281 non-null int64
marital_status    16281 non-null object
occupation        16281 non-null object
relationship      16281 non-null object
race              16281 non-null object
sex               16281 non-null object
capital_gain      16281 non-null int64
capital_loss      16281 non-null int64
hours_per_week    16281 non-null int64
native_country    16281 non-null object
dtypes: int64(6), object(8)
memory usage: 1.9+ MB


In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
# use scalar to unify the numberical columns
numerical_col = ["age", "fnlwgt", "education_num", "capital_gain","capital_loss", "hours_per_week"]

In [16]:
scaler = StandardScaler()
census_train[numerical_col] = scaler.fit_transform(census_train[numerical_col])
census_test[numerical_col] = scaler.transform(census_test[numerical_col])

In [17]:
census_train.columns

Index(['age', 'work_class', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [18]:
#notice the education is highly correlated to education_num, define the category columns to keep and drop 'education' in feature  
cat_feats = [ 'work_class', 'marital_status', 'occupation', 'relationship', 'race', 'sex','native_country']

In [19]:
X_train = census_train.drop(['education','income'], axis = 1)
y_train = census_train['income']
X_train = pd.get_dummies(X_train,columns=cat_feats,drop_first=True)
X_train.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,work_class_Local-gov,work_class_Never-worked,work_class_Private,work_class_Self-emp-inc,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
id,,,,,,,,,,,,,,,,,,,,,
1,0.030671,-1.063611,1.134739,0.148453,-0.21666,-0.035429,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.837109,-1.008707,1.134739,-0.145920,-0.21666,-2.222153,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,-0.042642,0.245079,-0.420060,-0.145920,-0.21666,-0.035429,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,1.057047,0.425801,-1.197459,-0.145920,-0.21666,-0.035429,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5,-0.775768,1.408176,1.134739,-0.145920,-0.21666,-0.035429,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
y_train.value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [21]:
X_test = census_test.drop(['education'], axis = 1)
X_test = pd.get_dummies(X_test,columns=cat_feats,drop_first=True)
X_test.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,work_class_Local-gov,work_class_Never-worked,work_class_Private,work_class_Self-emp-inc,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
id,,,,,,,,,,,,,,,,,,,,,
32562,-0.995706,0.350774,-1.197459,-0.145920,-0.21666,-0.035429,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
32563,-0.042642,-0.947095,-0.420060,-0.145920,-0.21666,0.774468,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
32564,-0.775768,1.394362,0.746039,-0.145920,-0.21666,-0.035429,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32565,0.397233,-0.279070,-0.031360,0.895083,-0.21666,-0.035429,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
32566,-1.508894,-0.817458,-0.031360,-0.145920,-0.21666,-0.845327,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


# Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [23]:
#column in X_train but not in X_train
X_train.columns.difference(X_test.columns)

Index(['native_country_Holand-Netherlands'], dtype='object')

In [24]:
# add a new column a new column and set value is 0
X_test['native_country_Holand-Netherlands'] = 0

In [25]:
rfc_pred = rfc.predict(X_test)

In [26]:
X_test['income'] = rfc_pred

In [27]:
X_test.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,work_class_Local-gov,work_class_Never-worked,work_class_Private,work_class_Self-emp-inc,...,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,native_country_Holand-Netherlands,income
id,,,,,,,,,,,,,,,,,,,,,
32562,-0.995706,0.350774,-1.197459,-0.145920,-0.21666,-0.035429,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
32563,-0.042642,-0.947095,-0.420060,-0.145920,-0.21666,0.774468,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
32564,-0.775768,1.394362,0.746039,-0.145920,-0.21666,-0.035429,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
32565,0.397233,-0.279070,-0.031360,0.895083,-0.21666,-0.035429,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
32566,-1.508894,-0.817458,-0.031360,-0.145920,-0.21666,-0.845327,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [28]:
#add 'income' column back to original copy of test data set
test['income'] = X_test['income'].apply(lambda x: '>50K' if x == 1 else '<=50K')
test.head()

,age,work_class,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
id,,,,,,,,,,,,,,,
32562,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
32563,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
32564,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,<=50K
32565,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
32566,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [29]:
# load existing data
file_name = 'Copy of censusTest.xlsx'
file_to_write = 'censusTest_output.xlsx'
sheet_df = pd.read_excel(file_name, index_col = 0)

#sheet_df.head()
sheet_df.reset_index(inplace=True)

# write and update
excel_writer = pd.ExcelWriter(file_to_write)
append_df = test[['income']]
append_df.reset_index(drop = True, inplace=True)
sheet_df = pd.concat([sheet_df, append_df], axis=1)
#sheet_df.head()

sheet_df.to_excel(excel_writer, 'Sheet1', index=False)
excel_writer.save()